# Automated ML

Importing all needed dependencies related to the project

In [1]:
import logging
import os
import json
import csv
import numpy as np
import pandas as pd
import pkg_resources
import joblib

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core import Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from sklearn.preprocessing import StandardScaler

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.27.0


In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl'

experiment=Experiment(ws, experiment_name)

In [15]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '3e42d11f-d64d-4173-af9b-12ecaa1030b3'
resource_group = 'aml-quickstarts-145360'
workspace_name = 'quick-starts-ws-145360'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='diabetes')
dataset.to_pandas_dataframe()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [18]:
dataset

{
  "source": [
    "('workspaceblobstore', 'UI/05-22-2021_055255_UTC/diabetes.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "df2f823a-fb29-4340-8aee-a0a741e89c08",
    "name": "diabetes",
    "version": 1,
    "workspace": "Workspace.create(name='quick-starts-ws-145360', subscription_id='3e42d11f-d64d-4173-af9b-12ecaa1030b3', resource_group='aml-quickstarts-145360')"
  }
}

In [9]:
#Display the first five records of the dataset
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [19]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for the cluster
cpu_cluster_name = "ComputeCluster"

# Verifing that cluster does not exist already or Create a New Cluster
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster.')
except ComputeTargetException:
    print('Creating a new compute cluster....')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

# get_status() to get a detailed status for the current running cluster 
print(compute_target.get_status().serialize())

Found existing cluster.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 2, 'targetNodeCount': 2, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 2, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-05-22T05:48:43.916000+00:00', 'errors': None, 'creationTime': '2021-05-22T05:46:50.202960+00:00', 'modifiedTime': '2021-05-22T05:47:05.548207+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 2, 'maxNodeCount': 5, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}


# Compute Cluster configuration for this project:

{'currentNodeCount': 2, 'targetNodeCount': 2, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 2, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-05-21T06:29:01.053000+00:00', 'errors': None, 'creationTime': '2021-05-21T06:26:56.693359+00:00', 'modifiedTime': '2021-05-21T06:27:12.055678+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 2, 'maxNodeCount': 5, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}

## AutoML Configuration

Automl settings and configuration used for this peoject:


In [20]:
# Automl settings 
automl_settings = {
    "experiment_timeout_minutes": 50,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5,
    "iterations": 30
    
}

# Automl configuration
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = 'classification',
                             training_data=dataset,
                             label_column_name='Outcome',
                             enable_early_stopping= True,
                             featurization = 'auto',
                             debug_log = 'automl_errors.log',
                             **automl_settings
                            )



In [21]:
# Submit experiment with AutoML Configuration

remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on ComputeCluster with default configuration
Running on remote compute: ComputeCluster


Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_203fb225-3062-46f0-be4a-b09778aff31a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.m

## Run Details

Now we will see best models for this classification problems.We can use the`RunDetails` widget to show various experiments performed.You can also see best model in ML Azure Portal.

In [22]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [23]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_203fb225-3062-46f0-be4a-b09778aff31a,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_203fb225-3062-46f0-be4a-b09778aff31a',
 'target': 'ComputeCluster',
 'status': 'Completed',
 'startTimeUtc': '2021-05-22T06:01:58.48634Z',
 'endTimeUtc': '2021-05-22T06:26:32.348926Z',
 'properties': {'num_iterations': '30',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'ComputeCluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl","subscription_id":"3e42d11f-d64d-4173-af9b-12ecaa1030b3","resource_group":"aml-quickstarts-145360","workspace_name":"quick-starts-ws-145360","region":"southcentralus","compute_target":"ComputeCluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"iterations":30,"primary_metric":"accuracy","task_type":"classification","data_s

In [24]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_203fb225-3062-46f0-be4a-b09778aff31a,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Best Model

Now we will see best model from the azure automl experiments.



In [25]:
# Retrieve and save best automl model
best_model, model = remote_run.get_output()
print(best_model)
print(model.steps)

Run(Experiment: automl,
Id: AutoML_203fb225-3062-46f0-be4a-b09778aff31a_28,
Type: azureml.scriptrun,
Status: Completed)
[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                feature_sweeping_config=None, feature_sweeping_timeout=None,
                featurization_config=None, force_text_dnn=None,
                is_cross_validation=None, is_onnx_compatible=None, logger=None,
                observer=None, task=None, working_dir=None)), ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('15',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('logisticregression',
                                                            Log

In [26]:
print(model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_weight_fraction_leaf=0.0,
                                                                                                  n_estimators=100,
                                                                                                  n_jobs=1,
                                        

In [27]:
print(best_model)

Run(Experiment: automl,
Id: AutoML_203fb225-3062-46f0-be4a-b09778aff31a_28,
Type: azureml.scriptrun,
Status: Completed)


In [28]:
remote_run.get_metrics()

{'experiment_status': ['DatasetEvaluation',
  'FeaturesGeneration',
  'DatasetFeaturization',
  'DatasetFeaturizationCompleted',
  'DatasetCrossValidationSplit',
  'ModelSelection',
  'BestRunExplainModel',
  'ModelExplanationDataSetSetup',
  'PickSurrogateModel',
  'EngineeredFeatureExplanations',
  'EngineeredFeatureExplanations',
  'RawFeaturesExplanations',
  'RawFeaturesExplanations',
  'BestRunExplainModel'],
 'experiment_status_description': ['Gathering dataset statistics.',
  'Generating features for the dataset.',
  'Beginning to fit featurizers and featurize the dataset.',
  'Completed fit featurizers and featurizing the dataset.',
  'Generating individually featurized CV splits.',
  'Beginning model selection.',
  'Best run model explanations started',
  'Model explanations data setup completed',
  'Choosing LightGBM as the surrogate model for explanations',
  'Computation of engineered features started',
  'Computation of engineered features completed',
  'Computation of ra

## Model Deployment

We have to deploy only one of the two models trained.. We will Perform the steps in the rest of this notebook only if we wish to deploy this model.

Now we have registered the model, created an inference config and deployed the model as a web service for endpoints consumption using professional API Testing tools (Swagger UI).

In [29]:
# Registring the best model
model = best_model.register_model(model_name='automl-diabetes-best-model',model_path='outputs/model.pkl')


In [30]:
# Now we will get environment with dependencies
environment = Environment.get(ws, "AzureML-AutoML")

In [32]:
inference_config = InferenceConfig(entry_script='scoring.py',
                                   environment=environment)
service_name = 'automl-deploy'
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,
                       overwrite=True
                      )
service.wait_for_deployment(show_output=True)

scoring_uri = service.scoring_uri
print(scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-22 06:34:35+00:00 Creating Container Registry if not exists.
2021-05-22 06:34:35+00:00 Registering the environment.
2021-05-22 06:34:37+00:00 Use the existing image.
2021-05-22 06:34:37+00:00 Generating deployment configuration.
2021-05-22 06:34:38+00:00 Submitting deployment to compute..
2021-05-22 06:34:41+00:00 Checking the status of deployment automl-deploy..
2021-05-22 06:39:07+00:00 Checking the status of inference endpoint automl-deploy.
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://69e25101-6ebf-4c1f-9585-72ba5329139e.southcentralus.azurecontainer.io/score


In [33]:
# Now we will try to Enable app insights
service.update(enable_app_insights=True)

Now we will consume the deployed webservice in AutoML so we are sending the web service requests to best model deployed

In [34]:
import requests
import json

#two set of data to score, so we get two results back
data = {"data": [{"Pregnancies": 9, 
     "Glucose": 130, 
     "BloodPressure": 60, 
     "SkinThickness": 20, 
     "Insulin": 6, 
     "BMI": 40.5, 
     "DiabetesPedigreeFunction": 0.537, 
     "Age": 50},

    {"Pregnancies": 8, 
     "Glucose": 70, 
     "BloodPressure": 80, 
     "SkinThickness": 30, 
     "Insulin": 18, 
     "BMI": 24.5, 
     "DiabetesPedigreeFunction": 0.951, 
     "Age": 38},
      ]}
    
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())
print("Case 0 : Not Diabetic, Case 1 : Diabetic.")

{"result": [1, 0]}
Case 0 : Not Diabetic, Case 1 : Diabetic.


Print the logs of the web service and delete the service

In [35]:
logs = service.get_logs()
logs

'2021-05-22T06:38:58,405636700+00:00 - gunicorn/run \n2021-05-22T06:38:58,406577500+00:00 - rsyslog/run \n2021-05-22T06:38:58,405643700+00:00 - iot-server/run \n2021-05-22T06:38:58,526032800+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_600883adcde8eb30a3a0fa1bdca2e09e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_600883adcde8eb30a3a0fa1bdca2e09e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_600883adcde8eb30a3a0fa1bdca2e09e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_600883adcde8eb30a3a0fa1bdca2e09e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_600883adcde8eb30a3a0fa1bdca2e09e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [32]:
#Deleting the deployed web service

service.delete()